In [1]:
import pandas as pd
import os
import FinanceDataReader as fdr
from tqdm import tqdm
import time, os
from multiprocessing import Pool

# 과제 1

- 종목코드

In [2]:
import pandas as pd
df_stock = pd.read_csv("raw_data.csv") #데이터 불러오기
df_stock['종목코드'] = df_stock['종목코드'].apply(lambda x : str(x).zfill(6))
lst_stock = df_stock.values.tolist()
# print(lst_stock [:10]) 

In [3]:
# dictionary 저장 후 파일 쓰기
dic_code2name = {}
for row in lst_stock:
    name, code, date = row[1], row[2], row[5]
    if date <= '2018-01-01': 
        dic_code2name[code] = name 

In [4]:
OF = open('assignment1_multi_processing.txt', 'w', encoding='UTF-8')
for cd in dic_code2name: 
    OF.write(f'{cd}\t{dic_code2name[cd]}\n')

# 과제 2 

In [5]:
def make_lst_result(code): 
    start_date = '20170101'
    end_date = '20211231'
    
    lst_date = []
    
    stock = fdr.DataReader(code, start = start_date, end = end_date)
    stock.reset_index(inplace=True)
  
    
    # dataframe -> list
    lst_stock = stock.values.tolist()
    
    
    for stock_row in lst_stock: 
        if stock_row[4] * stock_row[5] >= 100000000000: 
            lst_date.append(stock_row[0].date().strftime("%Y%m%d"))
        
    return [code, lst_date]

In [6]:
start_time = time.time()
num_cores = 10
pool = Pool(num_cores)
lst_code_date = pool.map(make_lst_result, dic_code2name.keys())
print(time.time() - start_time) 

17.84239411354065


In [8]:
OF = open("assignment2_multi_processing.txt", "w")
dic_result = {}

for code, lst_date  in lst_code_date:
    if lst_date == []:
        continue
    for date in lst_date:
        if code not in dic_result.keys(): 
            dic_result[code] = []
            dic_result[code].append(date)
        else: 
            dic_result[code].append(date)
        OF.write(f'{code}\t{date}\n')
            
OF.close()

# 과제 3

In [9]:
def make_dic_up(lst_code_date):
    start_date = '20170101' 
    end_date = '20211231' 
    code, lst_date = lst_code_date
    lst_result_up = []
    
    stock = fdr.DataReader(code, start = start_date, end = end_date)
    stock.reset_index(inplace=True)
    lst_stock = stock.values.tolist()  
    

    for idx, stock_val in enumerate(lst_stock):
        if (idx < 9) or (idx >= len(lst_stock)-1): 
            continue 
            
        date = stock_val[0].date().strftime("%Y%m%d")
        if date not in lst_date:
            continue
            
                                                
        D9_idx = idx-9
        next_idx = idx+1

        # 10일간의 데이터 (info)
        info_list = []
        for day_info in lst_stock[D9_idx: next_idx]:
            info_list += day_info[1:5] + [(day_info[4] * day_info[5])]
            
        info = ','.join(map(str, info_list))

        # D+1 종가 2% 상승 여부 (up) 
        up = int(lst_stock[next_idx][-1] >= 0.02)    

        lst_result_up.append([code, date, info, up])

    return lst_result_up

In [10]:
start_time = time.time()
num_cores = 20
pool = Pool(num_cores)
lst_code_date = pool.map(make_dic_up, lst_code_date)
print(time.time() - start_time) 

10.134265422821045


In [11]:
OF = open("assignment3_multi_processing_3.txt", 'w')

for row in lst_code_date: 
    for num in range(len(row)): 
        OF.write('\t'.join(map(str, row[num])) + '\n')
        
OF.close()